In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
data_dir = '../input/intel-image-classification'

valid_size = 0.2
batch_size = 32

random_transforms = [transforms.RandomRotation(25),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ColorJitter(0.5, 0.5, 0.5, 0.5)]
train_transforms = transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.RandomApply(random_transforms, p=0.4),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/seg_train/seg_train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/seg_test/seg_test', transform=test_transforms)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
validloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
output_classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [ ]:
data_iter = iter(trainloader)

images, labels = data_iter.next()
images = images.numpy()
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(32):
    ax = fig.add_subplot(2, 32/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(output_classes[labels[idx]])

In [ ]:
model = models.densenet121(pretrained=True)

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 512)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.1)),
                          ('fc2', nn.Linear(512, 256)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.1)),
                          ('fc3', nn.Linear(256, 6)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

is_cuda = torch.cuda.is_available()
print(is_cuda)

In [ ]:
criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

if is_cuda:
    model.cuda()

valid_loss_min = np.Inf
epochs = 10

for e in range(1, epochs+1):
    running_loss = 0
    valid_loss = 0.0
    model.train()
    for inputs, labels in trainloader:
        # Move input and label tensors to the device
        if is_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        
        # training
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()*inputs.size(0)
        
    model.eval()
    for data, target in validloader:
        # move tensors to GPU if CUDA is available
        if is_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    running_loss = running_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, running_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'intel_image_transfer.pt')
        valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load('intel_image_transfer.pt'))

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(6))
class_total = list(0. for i in range(6))

model.eval() # eval mode

# iterate over test data
for data, target in testloader:
    # move tensors to GPU if CUDA is available
    if is_cuda:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not is_cuda else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(data.size(0)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(5):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            output_classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (output_classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))